<a href="https://colab.research.google.com/github/JuanmaVL/SyS/blob/main/Copia_de_Parcial_2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente bloque de código instala varias bibliotecas necesarias para el proyecto:

- **`streamlit`**: Framework para crear aplicaciones web interactivas en Python.
- **`localtunnel`** (vía `npm`): Permite exponer un puerto local a la web.
- **`yt-dlp`**: Herramienta avanzada para descargar videos de YouTube y otras plataformas.
- **`pyngrok`**: Facilita la creación de túneles seguros con `ngrok`, útil para exponer servicios locales.
- **`sympy`**: Biblioteca para cálculos simbólicos en matemáticas.
- **`soundfile`**: Permite leer y escribir archivos de audio en distintos formatos.
- **`pytube`**: Otra biblioteca para descargar videos de YouTube.
- **`ffmpeg`**: Software para procesamiento y conversión de archivos de audio y video.
- **`pandas, gspread, oauth2client`**: Facilitan la manipulación de datos y la conexión con Google Sheets.
- **`numpy, matplotlib, scipy`**: Librerías científicas para análisis numérico y visualización.

Se reinstala `yt-dlp` desde su repositorio de GitHub para asegurar la versión más reciente.
Finalmente, se instalan todas las bibliotecas necesarias en una única línea para evitar posibles conflictos de dependencias.

In [1]:
!pip install streamlit -q
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok
!pip install sympy --upgrade
!pip install yt-dlp
!pip install soundfile
!pip install pytube
!pip install --upgrade pytube
!apt-get install ffmpeg
!pip install pandas gspread oauth2client
!pip install streamlit yt-dlp soundfile numpy matplotlib scipy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.5 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 6s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙Collecting https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
     - 2.7 MB 10.0 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.2.19-py3-none-any.whl size=2947620 sha256=21446df3d980dcce64c19dbf0a41b63aa7b94369033601140de42a8434e92475
  Stored in directory: /tmp/pip-ephem-wheel-cache-n39dvxhu/wheels/2d/79/97/7209650ef73114e0fe0603480da012ad3afacb9cae6b8acd9a
Successfully built yt-dlp
   ━━━━━━━━━

In [2]:
from pyngrok import ngrok

# Detener todos los túneles activos
ngrok.kill()

Este código en Python, utilizando Streamlit, implementa una aplicación interactiva para el análisis y reconstrucción de señales mediante la Serie de Fourier. La aplicación permite al usuario ingresar dos parámetros: la amplitud $( A )$ y la frecuencia $\ F_o $, que definen la señal de entrada dada por $\ x(t) = |A \cos(2\pi F_o t)|^2 $. Se genera un vector de tiempo adecuado para la señal y se grafica su representación en función del tiempo. Luego, el usuario puede seleccionar el número de armónicos para la reconstrucción de la señal utilizando un control deslizante. Se calculan los coeficientes de Fourier $\ c_n $, donde solo los términos de $\ n = 0, \pm 2 $ tienen valores distintos de cero, y se muestra el espectro en términos de magnitud y fase. Finalmente, se reconstruye la señal a partir de estos coeficientes y se compara con la señal original, mostrando el error de reconstrucción y permitiendo visualizar la mejora conforme aumenta el número de armónicos utilizados. El código genera tres gráficos interactivos: la señal original en función del tiempo, el espectro de Fourier (magnitud y fase) y la reconstrucción de la señal en comparación con la original. El usuario puede ajustar el número de armónicos en tiempo real para observar su impacto en la reconstrucción de la señal. Esta aplicación ha sido desarrollada con Python, Streamlit, NumPy y Matplotlib.


#Punto 1

In [3]:
%%writefile Punto1.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

# Configuración inicial
st.title("Análisis y Reconstrucción de Señales con Serie de Fourier")

# Parámetros de entrada
A = st.number_input("Ingrese un valor entero positivo para A:", min_value=1, value=5, step=1)
Fo = st.number_input("Ingrese un valor de frecuencia positiva para Fo:", min_value=0.1, value=1.0, step=0.1)

# Definir tiempos
ti = -1 / (2 * Fo)  # tiempo inicial
tf = 1 / (2 * Fo)   # tiempo final
T = tf - ti         # periodo total
wo = 2 * np.pi / T  # frecuencia angular
Fs = 30 * Fo  # frecuencia de muestreo

# Vector de tiempo
tv = np.arange(ti, tf, 1 / Fs)
Nm = len(tv)

# Señal de entrada
titulo_senal = "Señal de entrada $x(t) = |A \cos(2\pi F_o t)|^2$"
xo = abs(A * np.cos(2 * np.pi * Fo * tv))**2

# Gráfica de la señal de entrada
fig, ax = plt.subplots()
ax.plot(tv, xo, label='$x(t)$', color='b')
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('$x(t)$')
ax.set_title(titulo_senal)
ax.grid()
ax.legend()
st.pyplot(fig)

# Definir las bases de Fourier
N = st.slider("Seleccione el número de armónicos para la reconstrucción:", min_value=1, max_value=50, value=10)
phin = np.zeros((len(tv), 2 * N + 1), dtype=np.complex_)
for n in range(-N, N + 1):
    phin[:, n + N] = np.exp(1j * n * wo * tv)

# Cálculo de coeficientes de Fourier
cn = np.zeros(2 * N + 1, dtype=np.complex_)
nv = np.arange(-N, N + 1)
cn[N] = (A**2) / 2  # Coeficiente DC
cn[N + 2] = (A**2) / 4  # Armónico positivo
cn[N - 2] = (A**2) / 4  # Armónico negativo

# Gráfica del espectro de Fourier
fig, ax = plt.subplots(2, 1, figsize=(6, 6))
ax[0].stem(nv, abs(cn), 'r', markerfmt='ro')
ax[0].set_xlabel('n')
ax[0].set_ylabel('|Cn|')
ax[0].set_title("Espectro de Fourier - Magnitud")
ax[0].grid()

ax[1].stem(nv, np.angle(cn), 'r', markerfmt='ro')
ax[1].set_xlabel('n')
ax[1].set_ylabel('Fase (radianes)')
ax[1].set_title("Espectro de Fourier - Fase")
ax[1].grid()

st.pyplot(fig)

# Reconstrucción de la señal
Px = ((3 / 8) * A**4)  # Potencia de la señal
er = 1 - np.sum(abs(cn)**2) / Px  # Error de reconstrucción
xe = phin @ cn  # Señal reconstruida

# Gráfica de la reconstrucción
fig, ax = plt.subplots()
ax.plot(tv, xo, 'r', linewidth=2, label='$x(t)$ (Original)')
ax.plot(tv, np.real(xe), 'b--', linewidth=2, label='$x_e(t)$ (Reconstruida)')
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Amplitud')
ax.set_title(f"Reconstrucción con {N} armónicos (Error: {100*er:.2f}%)")
ax.legend()
ax.grid()
st.pyplot(fig)

st.write("Ajuste el número de armónicos para ver cómo cambia la reconstrucción.")


Writing Punto1.py


In [4]:
token="2tdroQQlWE22mpbWF56ZQwhCI6W_38v4DcENvFijbeGWZszYC"

In [5]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto1.py --server.port 5000 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5000', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://cc7c-34-169-137-250.ngrok-free.app


#PUNTO 2

Este script en Python implementa una aplicación web interactiva utilizando Streamlit, que permite la descarga, análisis y modulación de señales de audio. Para ello, el código ofrece dos opciones: cargar un archivo de audio local en formato MP3 o WAV, o descargar audio desde YouTube utilizando la biblioteca `yt-dlp`. Una vez obtenido el audio, la señal se visualiza y analiza en el dominio del tiempo y la frecuencia, mostrando su forma de onda y su espectro mediante la transformada de Fourier.

Posteriormente, se implementa un proceso de modulación de amplitud (AM), donde la señal original se multiplica por una portadora senoidal cuya frecuencia puede ajustarse mediante un control en la interfaz. Se permite la reproducción de la señal original, la señal modulada y la señal demodulada después de ser filtrada con un filtro paso bajo Butterworth, diseñado utilizando la biblioteca `scipy.signal`.

El código también emplea `soundfile` para la lectura y escritura de archivos de audio, `numpy` y `matplotlib` para el procesamiento y la graficación de las señales, y Streamlit para la creación de una interfaz interactiva. Esta aplicación es útil para visualizar los efectos de la modulación en señales de audio y comprender mejor los procesos involucrados en las telecomunicaciones.


In [6]:
%%writefile Punto2.py
import yt_dlp
import io
import soundfile as sf
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter as bw, filtfilt

# Función para descargar audio de YouTube
def download_audio_from_youtube(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}],
        'outtmpl': 'downloaded_audio.%(ext)s',
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return 'downloaded_audio.mp3'
    except Exception as e:
        st.error(f"Ocurrió un error al descargar el audio: {e}")
        return None

# Título de la aplicación
st.title("Análisis y Modulación de Audio")

# Elegir entre subir archivo o ingresar un link de YouTube
option = st.radio("Elige una opción:", ('Subir archivo', 'Ingresar link de YouTube'))

audio_data, fs = None, None

if option == 'Subir archivo':
    uploaded_file = st.file_uploader("Cargar un archivo de audio (mp3 o wav)", type=["mp3", "wav"])
    if uploaded_file is not None:
        audio_data, fs = sf.read(uploaded_file)
        st.audio(uploaded_file)

elif option == 'Ingresar link de YouTube':
    youtube_url = st.text_input("Introduce el enlace de YouTube")
    if youtube_url:
        file_path = download_audio_from_youtube(youtube_url)
        if file_path:
            audio_data, fs = sf.read(file_path)
            st.audio(file_path, format='audio/mp3')

# Si ya hay audio cargado (desde archivo o YouTube)
if audio_data is not None:
    st.write(f'Frecuencia de muestreo: {fs:.2f} Hz')

    # Selección del rango de tiempo
    ti = st.slider("Tiempo de inicio (s)", 0, int(len(audio_data) / fs) - 1, 5)
    tf = st.slider("Tiempo de finalización (s)", ti + 1, int(len(audio_data) / fs), 10)

    # Extraer señal
    m = audio_data[int(ti * fs):int(tf * fs)]
    if m.ndim > 1:
        m = m[:, 0]  # Convertir a mono si es estéreo

    # Reproducir señal de mensaje
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, m, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Graficar señal en el tiempo
    st.subheader("Señal en el Tiempo")
    tt = np.linspace(ti, tf, len(m))
    plt.figure()
    plt.plot(tt, m)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    st.pyplot(plt)

    # Transformada de Fourier
    Mw = np.fft.rfft(m)
    vf = np.fft.rfftfreq(len(m), 1/fs)

    # Graficar espectro
    st.subheader("Espectro de Frecuencia")
    plt.figure()
    plt.plot(vf, abs(Mw))
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    st.pyplot(plt)

    # Modulación AM
    Fo = st.number_input("Frecuencia de la portadora (Hz)", value=15000)
    A1 = st.number_input("Índice de modulación (A1)", value=2.0)
    c = np.cos(2 * np.pi * Fo * tt)
    y = A1 * m * c

    # Reproducir señal modulada
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, y, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Graficar señal modulada
    st.subheader("Señal Modulada en el Tiempo")
    plt.figure()
    plt.plot(tt, y)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    st.pyplot(plt)

    # Demodulación AM con filtrado
    d = y * c  # Multiplicación coherente
    N, Wn = 10, 14950  # Orden y frecuencia de corte
    num, den = bw(N, Wn, btype='low', fs=fs)
    xf = filtfilt(num, den, d)
    me = (2 / A1) * xf

    # Reproducir señal recuperada
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, me, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Graficar señal recuperada
    st.subheader("Señal Recuperada")
    plt.figure()
    plt.plot(tt, me)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    st.pyplot(plt)

    # Transformada de Fourier de la señal recuperada
    Mew = np.fft.rfft(me)
    st.subheader("Espectro de la Señal Recuperada")
    plt.figure()
    plt.plot(vf, abs(Mew))
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    st.pyplot(plt)

Writing Punto2.py


El código token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [7]:
token="2tdroQQlWE22mpbWF56ZQwhCI6W_38v4DcENvFijbeGWZszYC"

Este código permite ejecutar una aplicación de Streamlit y hacerla accesible desde Internet utilizando ngrok. Primero, se autentica con ngrok mediante un token de usuario, lo que permite gestionar túneles de forma segura. Luego, se ejecuta el archivo Punto2.py en un servidor local de Streamlit en el puerto 5029, usando nohup para que siga corriendo en segundo plano. Después, se establece un túnel con ngrok, permitiendo exponer el puerto de la aplicación a Internet mediante una URL pública. Finalmente, el código imprime el enlace generado por ngrok, que se puede utilizar para acceder a la aplicación desde cualquier navegador. Esta técnica es útil para compartir dashboards interactivos sin necesidad de abrir puertos manualmente o configurar servidores en la nube.

In [8]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto2.py --server.port 5029 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5029', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://b4c0-34-169-137-250.ngrok-free.app


# PUNTO 3

Este fragmento de código instala las librerías necesarias para ejecutar una aplicación en Streamlit y facilitar su acceso remoto.

- !pip install streamlit -q: Instala Streamlit, un framework en Python para crear aplicaciones web interactivas, con el argumento -q para que la instalación sea silenciosa.

- !npm install localtunnel: Instala localtunnel, una herramienta que permite exponer servidores locales a Internet sin necesidad de configurar redes o abrir puertos.

- !python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz: Descarga e instala yt-dlp directamente desde su repositorio de GitHub, asegurando una versión actualizada mediante --force-reinstall. Esta biblioteca permite descargar contenido multimedia de plataformas como YouTube.

- !pip install pyngrok: Instala pyngrok, una interfaz en Python para ngrok, que
permite crear túneles seguros para exponer servidores locales a Internet, facilitando el acceso remoto a la aplicación de Streamlit.

In [9]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧Collecting https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.2.19-py3-none-any.whl size=2947620 sha256=21446df3d980dcce64c19dbf0a41b63aa7b94369033601140de42a8434e92475
  Stored in directory: /tmp/pip-ephem-wheel-cache-0i9av018/wheels/2d/79/97/7209650ef73114e0fe0603480da012ad3afacb9cae6b8acd9a
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2025.2.19
    Uninstalling yt-dlp-2025.2.19:
      Successfully uninstalled yt-dlp-20

Este código implementa una aplicación interactiva en Streamlit para analizar el Total Harmonic Distortion (THD) en rectificadores de onda completa con cargas resistivas y RC. Para ello, genera una señal rectificada de onda completa y calcula su Transformada Rápida de Fourier (FFT) con el fin de analizar su contenido espectral. A partir de la FFT, se calcula el THD, que mide la distorsión armónica total de la señal considerando la relación entre los armónicos superiores y el primer armónico.

La aplicación permite al usuario ajustar la frecuencia de entrada de la señal, así como los valores de la resistencia (R) y la capacitancia (C) de la carga mediante deslizadores interactivos. Posteriormente, se muestran gráficos de la señal en el dominio del tiempo, su espectro de frecuencia (FFT) y los valores de THD obtenidos para cada tipo de carga. Este análisis es especialmente útil en el estudio de circuitos eléctricos y electrónicos, permitiendo evaluar el impacto de diferentes cargas en la distorsión de la señal de salida en rectificadores.

In [10]:
%%writefile Punto3.py
import streamlit as st
from pyngrok import ngrok
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
# Título y descripción general
st.title('Cálculo del THD en Circuitos RC y Resistivos Puros')
st.write("""
En este dashboard se muestra el cálculo de la **Distorsión Armónica Total (THD)** en circuitos RC y resistivos puros.
""")

st.write("""
### ¿Qué es el THD?
La **Distorsión Armónica Total (THD)** es una medida de la cantidad de distorsión armónica presente en una señal.
Se calcula comparando la magnitud de los armónicos superiores con la magnitud de la frecuencia fundamental.

El **THD** se define como:
""")

st.latex("THD = \\frac{\\sqrt{V_2^2 + V_3^2 + V_4^2 + ...}}{V_1}")

st.write("""
Donde: """)
st.latex("V_1")
st.write("""es la magnitud de la frecuencia fundamental y: """)
st.latex("V_2, V_3, V_4,...")
st.write("""son las magnitudes de los armónicos superiores.
""")

# Explicación del proceso de cálculo del THD usando FFT
st.write("""
### Cálculo del THD a partir de la FFT
Para calcular el THD, se uasa la **Transformada Rápida de Fourier (FFT)**, que permite transformar una señal del dominio del tiempo al dominio de la frecuencia.

Primero, se obtiene la **frecuencia fundamental** de la señal, que es el componente de mayor magnitud en el espectro de frecuencias. Luego, se calcula la contribución de los armónicos (frecuencias superiores) y con la fórmula anterior se obtiene el THD.
""")

# Parámetros de la señal de entrada
Fo = 60  # Frecuencia de la señal (Hz)
Fs = 30 * Fo  # Frecuencia de muestreo (Hz)
To = 1 / Fo  # Periodo fundamental
Ts = 1 / Fs  # Periodo de muestreo
t = np.arange(0, 5 * To, Ts)  # Simulamos 5 periodos de la señal
A = 120  # Amplitud de la señal de entrada

# Generación de la señal de entrada y su rectificación
st.subheader("Señal de Entrada y Rectificación")
st.write("""
Se genera una señal senoidal de 60 Hz, que es típica en sistemas de alimentación eléctrica.
Luego, se rectifica para simular la señal que pasa por un circuito de rectificación.
""")
in_o = A * np.sin(2 * np.pi * Fo * t)
rec_c = sig.square(2 * np.pi * Fo * t)  # Tren de pulsos para la señal rectificada completa
in_ = in_o * rec_c  # Entrada rectificada completa

# Mostrar las gráficas de la señal de entrada y rectificada
st.write("### Señal de Alimentación Original (Senoidal)")
plt.figure()
plt.plot(t, in_o, label='Alimentación')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)

st.write("### Señal Rectificada Completa")
plt.figure()
plt.plot(t, in_, label='Señal Rectificada')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)

# Slider para ajustar R y C
st.subheader("Parámetros del Circuito RC")
st.write("""
Ahora se pueden modificar los valores de la resistencia (R) y la capacitancia (C) para observar cómo afecta el circuito RC a la señal rectificada.
""")
R_v = st.slider('Resistencia R (Ohmios)', min_value=100, max_value=2000, value=1000, step=100)
C_v = st.slider('Capacitancia C (μF)', min_value=1, max_value=50, value=10, step=1) * 1e-6

# Función de transferencia del circuito RC en serie
st.write("""
### Función de Transferencia del Circuito RC en Serie
Para el circuito RC en serie, la función de transferencia se define como:
""")
st.latex("H(s) = \\frac{1}{RCs + 1}")


st.write("""
Esta función de transferencia permite modelar el comportamiento del circuito en el dominio de la frecuencia.
""")
# Definición de la función de transferencia del circuito RC
num = np.array([1])  # Coeficiente del numerador
den = np.array([R_v * C_v, 1])  # Coeficientes del denominador
G_n = sig.TransferFunction(num, den)  # Definición de la función de transferencia

# Evaluar la salida del circuito RC
_, out_rc, _ = sig.lsim(G_n, U=in_, T=t)

# Gráfica de la salida del circuito RC
st.subheader("Salida del Circuito RC")
st.write("""
La siguiente gráfica muestra cómo la señal rectificada cambia al pasar por el circuito RC en serie.
""")
plt.figure()
plt.plot(t, out_rc, label='Salida del Circuito RC')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)
#plt.plot(t, in_,label='In(t)')#entrada rectificada
st.subheader("Salida del Circuito resistivo")
st.write("""
La siguiente gráfica muestra cómo la señal rectificada de salida del circuito resistivo.
""")
plt.figure()
plt.plot(t, in_,c="r",label='Out(t)')#salida del circuito
plt.xlabel('t [s]')
plt.ylabel('out(t)')
st.pyplot(plt)
# Cálculo del THD simplificado basado en la FFT
def calcular_thd(signal, Fs):
    Xf = np.fft.rfft(signal)  # FFT de la señal
    magnitudes = np.abs(Xf)
    freqs = np.fft.rfftfreq(len(signal), 1 / Fs)

    # Identificar la frecuencia fundamental
    fundamental_idx = np.argmax(magnitudes)
    fundamental = magnitudes[fundamental_idx]

    # Cálculo del THD como la suma de los armónicos sobre la fundamental
    thd = np.sqrt(np.sum(magnitudes[1:]**2)) / fundamental
    return thd

# Calcular el THD
thd_rc = calcular_thd(out_rc, Fs)
thd_resistive = calcular_thd(in_, Fs)

# Mostrar el valor del THD
st.subheader("Resultados del THD")
st.markdown("""
El valor del THD es una medida de la distorsión de la señal debida a los armónicos.
A continuación, se muestran los valores del THD tanto para el circuito resistivo puro como para el circuito RC en serie:
""")
st.markdown(f"**THD del circuito RC**: {thd_rc * 100:.2f}%")
st.markdown(f"**THD del circuito resistivo puro**: {thd_resistive * 100:.2f}%")
st.subheader('Cálculo del Factor de Potencia basado en THD')

st.subheader('¿Qué es el Factor de Potencia?')
st.write('''
El **Factor de Potencia (FP)** mide la eficiencia con la que un sistema eléctrico convierte la energía eléctrica en trabajo útil.
Cuando la señal tiene **Distorsión Armónica Total (THD)**, parte de la potencia se pierde en los armónicos, reduciendo la eficiencia.

La fórmula general del Factor de Potencia con THD es:
''')

st.latex(r'''
FP = \sqrt{\frac{1}{1 + \left(THD\right)^2}}
''')

st.write('''
Donde el **THD** representa el porcentaje de distorsión de la señal. A medida que aumenta el **THD**, más potencia se pierde en los armónicos, reduciendo el **Factor de Potencia**.
''')
st.markdown("""
A continuación, se muestran los valores del factor de potencia tanto para el circuito resistivo puro como para el circuito RC en serie:
""")
pf1 = np.sqrt(1 / (1 + (thd_rc )**2))
pf2 = np.sqrt(1 / (1 + (thd_resistive)**2))
st.markdown(f"**FP del circuito RC**: {pf1:.4f}")
st.markdown(f"**FP del circuito resistivo puro**: {pf2:.4f}")

# FFT de la señal de salida para el circuito RC
vfre_rc = np.fft.rfftfreq(len(out_rc), Ts)  # Vector de frecuencias en Hz
Xf_rc = np.fft.rfft(out_rc)  # FFT de la salida

st.markdown("### FFT de la Salida del Circuito RC")
st.markdown("""
La **Transformada de Fourier** nos muestra el contenido de frecuencias de la señal.
La siguiente gráfica muestra cómo se distribuyen las frecuencias en la señal después de pasar por el circuito RC.
""")
plt.figure()
plt.stem(vfre_rc, np.abs(Xf_rc) / len(out_rc))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud FFT')
st.pyplot(plt)

# FFT de la señal para el circuito resistivo puro
vfre_resistive = np.fft.rfftfreq(len(in_), Ts)
Xf_resistive = np.fft.rfft(in_)

st.write("### FFT del Circuito Resistivo Puro")
st.write("""
Este gráfico muestra la FFT de la señal en un circuito resistivo puro, sin ningún circuito adicional que altere la señal.
""")
plt.figure()
plt.stem(vfre_resistive, np.abs(Xf_resistive) / len(in_))
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud FFT')
st.pyplot(plt)

Writing Punto3.py


El código token="2lmAC6DGxB9xF06bq6Js7K7IZQK_5T6aE4rBZBp6hDqefow8M" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [11]:
token="2tdroQQlWE22mpbWF56ZQwhCI6W_38v4DcENvFijbeGWZszYC"

Este código permite ejecutar una aplicación de Streamlit y hacerla accesible desde Internet utilizando ngrok. Primero, se autentica con ngrok mediante un token de usuario, lo que permite gestionar túneles de forma segura. Luego, se ejecuta el archivo Punto3.py en un servidor local de Streamlit en el puerto 5039, usando nohup para que siga corriendo en segundo plano. Después, se establece un túnel con ngrok, permitiendo exponer el puerto de la aplicación a Internet mediante una URL pública. Finalmente, el código imprime el enlace generado por ngrok, que se puede utilizar para acceder a la aplicación desde cualquier navegador. Esta técnica es útil para compartir dashboards interactivos sin necesidad de abrir puertos manualmente o configurar servidores en la nube.

In [12]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto3.py --server.port 5039 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5039', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://79cb-34-169-137-250.ngrok-free.app
